In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os, glob

In [2]:
path = os.path.join(os.path.dirname(os.getcwd()), '../Data/Extracted_data/')
os.chdir(path)

### MBN

In [19]:
MBN = pd.read_csv('MBN.csv', index_col='Sample')
MBN

,location,measure,peak,peak_ref,fwhm,fwhm_ref,diff_two_peaks,diff_two_peaks_ref,amp,amp_ref,pos,pos_ref,energy,energy_ref
Sample,,,,,,,,,,,,,,
41C,1,1,0.267420,0.164482,0.033634,0.019858,0.035758,0.025778,0.015838,0.013743,7686.0,7624,408.764909,273.907729
41C,1,2,0.220200,0.164482,0.029495,0.019858,0.052288,0.025778,0.017809,0.013743,7642.0,7624,407.653684,273.907729
41C,1,3,0.222581,0.164482,0.028561,0.019858,0.048347,0.025778,0.014434,0.013743,7638.0,7624,395.134422,273.907729
41C,2,1,0.269990,0.164482,0.034059,0.019858,0.050624,0.025778,0.018987,0.013743,7677.0,7624,390.800935,273.907729
41C,2,2,0.246733,0.164482,0.033052,0.019858,0.060581,0.025778,0.020128,0.013743,7642.0,7624,415.882398,273.907729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6C,5,2,0.266634,0.177791,0.030577,0.016124,0.055583,0.025268,0.014134,0.010430,7588.0,7684,356.821249,249.070282
6C,5,3,0.219244,0.177791,0.022055,0.016124,0.037694,0.025268,0.012990,0.010430,7604.0,7684,326.323321,249.070282
6C,6,1,0.194222,0.177791,0.024528,0.016124,0.042488,0.025268,0.013571,0.010430,7574.0,7684,337.986332,249.070282


In [20]:
MBN = MBN.groupby("Sample").mean()

MBN['peak_ratio'] = MBN['peak']/MBN['peak_ref']
MBN['fwhm_ratio'] = MBN['fwhm']/MBN['fwhm_ref']
MBN['diff_two_peaks_ratio'] = MBN['diff_two_peaks']/MBN['diff_two_peaks_ref']
MBN['amp_ratio'] = MBN['amp']/MBN['amp_ref']
MBN['pos_ratio'] = MBN['pos']/MBN['pos_ref']
MBN['energy_ratio'] = MBN['energy']/MBN['energy_ref']

MBN = MBN.drop(columns=['location', 'measure', 
                        'peak', 'peak_ref', 
                        'fwhm', 'fwhm_ref', 
                        'diff_two_peaks', 'diff_two_peaks_ref',
                        'amp', 'amp_ref',
                        'pos', 'pos_ref',
                        'energy', 'energy_ref'])
MBN

,peak_ratio,fwhm_ratio,diff_two_peaks_ratio,amp_ratio,pos_ratio,energy_ratio
Sample,,,,,,
10C,0.333814,0.277020,0.230009,1.257148,0.992710,0.457476
13C,1.055631,0.902723,1.163056,1.237837,1.002948,1.109720
14C,1.500043,1.565899,1.677959,0.939024,0.966947,1.755207
16C,0.897266,0.798002,0.684264,0.927139,0.987522,0.910617
19C,1.090518,0.947283,0.763296,1.128928,1.002407,1.126849
20C,0.797447,0.917009,0.499761,0.975842,0.995773,0.981373
21C,0.839725,0.692852,0.705402,1.151889,0.984968,0.853808
23C,0.829439,0.692435,0.397580,0.919473,0.999861,0.846653
24C,1.209868,0.869175,0.588694,1.061223,1.005517,1.030618


### NLE

In [21]:
NLE = pd.read_csv('NLE.csv')
NLE['NLE_avg'] = NLE[['S10', 'S11', 'S12']].mean(axis=1)
NLE = pd.pivot_table(NLE, index='Sample' ,columns='freq', values='NLE_avg')
NLE = NLE.add_prefix('NLE_avg_')
NLE[['NLE_ratio_51', 'NLE_ratio_85', 'NLE_ratio_119']] = NLE[['NLE_avg_51', 'NLE_avg_85', 'NLE_avg_119']].div(NLE['NLE_avg_17'], axis=0)
NLE = NLE[['NLE_avg_51', 'NLE_avg_85', 'NLE_avg_119']]
NLE.head()

freq,NLE_avg_51,NLE_avg_85,NLE_avg_119
Sample,,,
10C,0.418210,0.512854,0.228678
13C,0.425272,0.509204,0.216304
14C,0.411063,0.491542,0.215602
16C,0.403479,0.472269,0.197359
19C,0.460072,0.580688,0.249148


### Electric

In [22]:
electric = pd.read_csv('Electric_RP.csv', index_col='ID')
electric.drop('Unnamed: 0', axis=1, inplace=True)
electric = electric[['Avg_RP', 'ACEE_30_Avg_58', 'ACEE_500_Avg_58', 'ACPD_Avg_58']]
electric.head()

,Avg_RP,ACEE_30_Avg_58,ACEE_500_Avg_58,ACPD_Avg_58
ID,,,,
41C,190.979294,-0.137640,-2.866030,4.804749
43C,184.930215,-0.462040,-2.254125,3.869428
44C,187.204261,-0.596385,-2.788500,3.370729
45C,189.010633,-0.645710,-3.202220,4.186639
7C,179.123964,-1.135960,-1.718115,3.438451


### NLO

In [23]:
NLO = pd.read_csv('NLO_result.csv', index_col='Sample')
NLO.drop(['Stress', 'St Dev','Confidence'], axis=1, inplace=True)
NLO.rename(columns={'Average':'NLO_avg'}, inplace=True)
NLO.head()

,NLO_avg
Sample,
44C,4.619563
41C,5.576476
13C,10.196998
13C,9.082132
13C,9.784843


### Fatigue

In [24]:
fatigue = pd.read_excel('FatigueSampleInventory.xlsx', 
                        index_col=' Specimen ID',
                        usecols=[' Specimen ID', 'Cycles at NDE Test', 'Cycles at Failure', 'Current Unfailed Cycles']
                       )
fatigue.rename(columns={' Specimen ID':'ID', 'Cycles at NDE Test':'NDE_cyle', 'Cycles at Failure':'failure_cycle', 'Current Unfailed Cycles':'unfailed_cycle'}, inplace=True)
fatigue['f'] = fatigue['NDE_cyle']/fatigue['failure_cycle']
fatigue.head()

,NDE_cyle,failure_cycle,unfailed_cycle,f
Specimen ID,,,,
41C,0,2154137.0,NaN,0.0
43C,0,NaN,NaN,NaN
44C,0,1318034.0,NaN,0.0
45C,0,NaN,3245267.0,NaN
7C,150000,NaN,NaN,NaN


Merge all data

In [25]:
merged = fatigue.join([MBN, NLE, NLO, electric], how='left')

In [26]:
merged.to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Merged_data/MERGE_on_sample.csv'))

In [27]:
merged

,NDE_cyle,failure_cycle,unfailed_cycle,f,peak_ratio,fwhm_ratio,diff_two_peaks_ratio,amp_ratio,pos_ratio,energy_ratio,NLE_avg_51,NLE_avg_85,NLE_avg_119,NLO_avg,Avg_RP,ACEE_30_Avg_58,ACEE_500_Avg_58,ACPD_Avg_58
10C,300000,NaN,NaN,NaN,0.333814,0.277020,0.230009,1.257148,0.992710,0.457476,0.418210,0.512854,0.228678,4.240700,171.622934,-0.797640,-1.795000,2.579638
13C,150000,NaN,NaN,NaN,1.055631,0.902723,1.163056,1.237837,1.002948,1.109720,0.425272,0.509204,0.216304,10.196998,170.130491,-0.660875,-2.167395,3.725514
13C,150000,NaN,NaN,NaN,1.055631,0.902723,1.163056,1.237837,1.002948,1.109720,0.425272,0.509204,0.216304,9.082132,170.130491,-0.660875,-2.167395,3.725514
13C,150000,NaN,NaN,NaN,1.055631,0.902723,1.163056,1.237837,1.002948,1.109720,0.425272,0.509204,0.216304,9.784843,170.130491,-0.660875,-2.167395,3.725514
14C,2000000,NaN,NaN,NaN,1.500043,1.565899,1.677959,0.939024,0.966947,1.755207,0.411063,0.491542,0.215602,4.588220,159.347604,-0.828250,-0.828250,3.291392
16C,2000000,NaN,NaN,NaN,0.897266,0.798002,0.684264,0.927139,0.987522,0.910617,0.403479,0.472269,0.197359,3.755165,169.558497,-1.108970,-1.808550,2.446152
19C,450000,914392.0,NaN,0.492130,1.090518,0.947283,0.763296,1.128928,1.002407,1.126849,0.460072,0.580688,0.249148,3.798736,171.925123,-1.065155,-1.570740,3.226957
20C,2000000,NaN,NaN,NaN,0.797447,0.917009,0.499761,0.975842,0.995773,0.981373,0.407136,0.490481,0.215616,3.791726,170.021810,-1.223960,-1.223960,2.646820
21C,600000,800007.0,NaN,0.749993,0.839725,0.692852,0.705402,1.151889,0.984968,0.853808,0.423233,0.509838,0.224605,4.626314,174.003954,-0.884900,-1.578795,2.764862
23C,600000,NaN,2855225.0,NaN,0.829439,0.692435,0.397580,0.919473,0.999861,0.846653,0.419344,0.508837,0.226340,5.621833,173.403495,-0.831110,-1.194540,3.061743
